# Precision-cancer

Packages:
Numpy version: 1.20.0;
Pandas version: 1.2.1;
scikit-learn version: 0.24.1;
lifelines version: 0.27.8;
scikit-survival version: 0.22.2: 

In [1]:
import pandas as pd
import numpy as np
from lifelines import CoxPHFitter, KaplanMeierFitter
from sklearn.linear_model import LogisticRegression
from sksurv.ensemble import RandomSurvivalForest
import warnings
warnings.simplefilter(action='ignore')

## Load Synthetic Data

For this notebook, we use synthetic data in "synthetic_data.csv" as a demonstration of our pipeline. It includes the following features -
- Confounders. We note that here is one toy example. In real applications, more confounders and more categories should be included.
    - <font color=darkblue>*Gender*</font>: whether the patient is female (=1) or not (=0).
    - <font color=darkblue>*Age*</font>: whether age >= 40 (=1) or not (=0).
    - <font color=darkblue>*ECOG*</font>: whether ECOG >=3 (=1) or not (=0).
- <font color=darkblue>*Mutation status*</font> for Gene A, Gene B, Gene C, Gene D, and Gene E.
    - Whether the gene is mutated (=1) or not (=0).
- <font color=darkblue>*Treatment status*</font> for Treatment 1 and Treatment 2.
    - Whether the patient received the treatment (=1) or not (=0).
- Survival information
    - <font color=darkblue>*duration*</font>: time from start of treatment to event (days).
    - <font color=darkblue>*event*</font>: death happened (=1) or right censored (=0)
    - <font color=darkblue>*fmi_test*</font>: time from start of treatment to FMI test (days).

In [2]:
data_cox = pd.read_csv('synthetic_data.csv')
print('Number of patients: %d' % data_cox.shape[0])
cols_basic = ['duration', 'event', 'fmi_test'] # Basic infomation for survival analysis
confounders = ['Female', 'Age >= 40', 'ECOG >= 3'] # Confounders
data_cox.head()

Number of patients: 5000


,Female,Age >= 40,ECOG >= 3,Gene A,Gene B,Gene C,Gene D,Gene E,Treatment 1,Treatment 2,duration,event,fmi_test
0,1,1,0,1,0,1,0,1,0,1,72,1,8
1,1,1,1,1,0,1,1,1,0,1,33,0,5
2,1,1,1,0,0,1,1,1,1,0,41,1,28
3,0,0,1,1,0,1,0,0,0,1,343,0,9
4,0,0,1,0,1,0,1,0,1,0,126,1,27


## Analysis

In [3]:
def process_HR(cph_summary, name_feature):
    '''
    Process result of CoxPH model to HR with 95% confidence intervals.
    '''
    result = cph_summary.loc[name_feature, ['exp(coef)', 'exp(coef) lower 95%', 'exp(coef) upper 95%']].values
    HR = '%.2f (%.2f, %.2f)' % tuple(result)   
    return HR
    
def Cox_IPTW(data_fit, name_feature):
    '''
    Uni-covariate CoxPH model with IPTW and left-truncation. 
    name_feature is the feature we are interested in (e.g., gene name or treatment name).
    Return the HR for name_feature.
    '''
    # Generate data used in Cox Regression with IPTW Weights
    data_confounders = data_fit.iloc[:, 3:].copy().drop(columns=[name_feature])
    df = pd.concat([data_fit.iloc[:, :3], data_fit[[name_feature]]], axis=1)
    X = data_confounders.values
    y = data_fit[name_feature].values
    model = LogisticRegression(solver='lbfgs', n_jobs=-1, class_weight='balanced')
    model.fit(X, y)
    p_treated = float(np.sum(y==1))/y.shape[0]
    propensity_scores = model.predict_proba(X)[:, 1]
    df['weights'] = 0
    IP_treated = 1 / propensity_scores
    IP_untreated = 1 / (1 - propensity_scores)        
    df.loc[df[name_feature]==1, 'weights'] = IP_treated[df[name_feature]==1]
    df.loc[df[name_feature]==0, 'weights'] = IP_untreated[df[name_feature]==0]    
    # Cox Regression
    cph = CoxPHFitter()
    cph.fit(df, 'duration', 'event', weights_col='weights', entry_col='fmi_test', robust=True)
    HR = process_HR(cph.summary, name_feature)
    return HR

### 1. Association between mutations and patient survival.

Here we study the prognostic effects of mutations in Gene A on overall survival (OS) and report the OS HR.

In [4]:
data_fit = data_cox[cols_basic+confounders+['Gene A']]
HR = Cox_IPTW(data_fit, 'Gene A')
print('OS HR for Gene A: %s' % (HR))

OS HR for Gene A: 1.10 (1.03, 1.17)


### 2. Gene-treatment interactions

Here we use analyze the interaction between mutations of Gene A and Treatment 1.

In [5]:
gene = 'Gene A'
treatment = 'Treatment 1'

##### Interaction

Interaction: exp(coef) for the drug-treatment interaction term in the Cox model, adjusted by patient confounders.

In [6]:
data_fit = data_cox[cols_basic+[gene, treatment]]
col_inter = '%s+%s'%(gene, treatment)
data_fit[col_inter] = data_fit[gene]*data_fit[treatment]
cph = CoxPHFitter()
cph.fit(data_fit, 'duration', 'event', entry_col='fmi_test', robust=True)
HR = process_HR(cph.summary, col_inter)
print('Interaction between Gene A and Treatment 1 is %s' % HR)

Interaction between Gene A and Treatment 1 is 0.68 (0.60, 0.77)


##### Gene-centric
- <font color=darkblue>*HR for gene mutation*</font>: for all patients with mutations of Gene A, OS HR for patients who received Treatment 1 vs. patients who did not take Treatment 1.
- <font color=darkblue>*HR for absence of gene mutation*</font>: for all patients *without* mutations of Gene A, OS HR for patients who received Treatment 1 vs. patients who did not take Treatment 1.

In [7]:
data_fit = data_cox.loc[data_cox[gene]==1, cols_basic+confounders+[treatment]].copy()
HR = Cox_IPTW(data_fit, treatment)
print('Treatment HR for patients with mutations of Gene A = %s' % HR)

data_fit = data_cox.loc[data_cox[gene]==0, cols_basic+confounders+[treatment]].copy()
HR = Cox_IPTW(data_fit, treatment)
print('Treatment HR for patients without mutations of Gene A = %s' % HR)

Treatment HR for patients with mutations of Gene A = 0.76 (0.69, 0.84)
Treatment HR for patients without mutations of Gene A = 1.15 (1.06, 1.25)


##### Drug-centric

- <font color=darkblue>*HR given drug*</font>: for all patients who received Treatment 1, HR for patients with mutations of Gene A vs. patients without mutations of Gene A.
- <font color=darkblue>*HR given other drugs*</font>: for all patients who received other treatments, HR for patients with mutations of Gene A vs. patients without mutations of Gene A.

In [8]:
data_fit = data_cox.loc[data_cox[treatment]==1, cols_basic+confounders+[gene]].copy()
HR = Cox_IPTW(data_fit, gene)
print('Gene A mutation HR for patients who received Treatment 1 = %s' % HR)

data_fit = data_cox.loc[data_cox[treatment]==0, cols_basic+confounders+[gene]].copy()
HR = Cox_IPTW(data_fit, gene)
print('Gene A mutation HR for patients who didn\'t receive Treatment 1 = %s' % HR)

Gene A mutation HR for patients who received Treatment 1 = 0.92 (0.85, 1.01)
Gene A mutation HR for patients who didn't receive Treatment 1 = 1.37 (1.25, 1.51)


### 3. Pathway-mutation interactions

Here we study the pathway-mutation interactions. A pathway was considered mutated if any gene within that pathway was mutated. Here we suppose Pathway 1 corresponds to Gene A, Gene B, and Gene C.


In [9]:
pathway = 'Pathway 1'
genes_pathway = ['Gene A', 'Gene B', 'Gene C']

data_fit = data_cox.copy()
data_fit[pathway] = (data_fit[genes_pathway].sum(axis=1) > 0).astype(int)

data_fit = data_fit[cols_basic+[pathway, treatment]]
col_inter = '%s+%s'%(pathway, treatment)
data_fit[col_inter] = data_fit[pathway]*data_fit[treatment]
cph = CoxPHFitter()
cph.fit(data_fit, 'duration', 'event', entry_col='fmi_test', robust=True)
HR = process_HR(cph.summary, col_inter)
print('Interaction between %s and Treatment 1 is %s' % (pathway, HR))

Interaction between Pathway 1 and Treatment 1 is 0.59 (0.45, 0.76)


### 4. RSF score for treatment

Here we compute the binary RSF score for treatment 1. 

We developed the Random Survival Forest (RSF) algorithm to predict patient survival time from the patient genomics profile. 

The binary RSF score is 1 (i.e. high) if the predicted survival time with given treatment is longer than the predicted survival time without the given treatment for each patient, and 0 (i.e. low) if vice versa.

In [10]:
def generate_RSF_data(data_cox, genes):
    '''
    Transform the data into the form to feed into the random survival forest:
        X: mutation status profile (genes)
        y: survival outcome
    '''
    # We predict the survival time only from the mutation status
    X = data_cox[genes].values
    y = [] 
    for idx in data_cox.index:
        event = True if data_cox.loc[idx, 'event']==1 else False
        y.append((event, data_cox.loc[idx, 'duration']))
    y = np.array(y, dtype=[('cens', '?'), ('time', '<f8')])
    y = np.array(y)
    return X, y


def fit_RSF(data_cox, genes):
    '''
    Fit RSF model to predict survival time from the mutation status of genes.
    '''
    X, y = generate_RSF_data(data_cox, genes)
    rsf = RandomSurvivalForest(n_estimators=20,
                               min_samples_split=10,
                               min_samples_leaf=15, 
                               max_features="sqrt",
                               n_jobs=-1,
                               random_state=1001)
    rsf.fit(X, y)
    return rsf

def predict_RSF(data_cox, genes, rsf):
    '''
    Retrieve the predicted survival time, given mutation status of genes and RSF model rsf.
    '''
    X, y = generate_RSF_data(data_cox, genes)
    return rsf.predict(X)


treatment = 'Treatment 1'
genes = ['Gene A', 'Gene B', 'Gene C', 'Gene D', 'Gene E']

# Train-test split
train_size = 0.7
np.random.seed(1001)
index_train = np.random.choice(data_cox.index, int(train_size*data_cox.shape[0]), replace=False)
data_fit_train = data_cox.loc[index_train].copy()
data_fit_test = data_cox.loc[~data_cox.index.isin(index_train)].copy()

# Fit RSF model
rsf_treatment = fit_RSF(data_fit_train.loc[data_fit_train[treatment]==1], genes)
rsf_nontreatment = fit_RSF(data_fit_train.loc[data_fit_train[treatment]==0], genes)

# On test data, compute the survival time
time_treatment = predict_RSF(data_fit_test, genes, rsf_treatment)
time_nontreatment = predict_RSF(data_fit_test, genes, rsf_nontreatment)

# Compute the binary RSF score
data_fit_test['RSF Score'] = (time_treatment < time_nontreatment).astype(int)

data_fit_test.head()

,Female,Age >= 40,ECOG >= 3,Gene A,Gene B,Gene C,Gene D,Gene E,Treatment 1,Treatment 2,duration,event,fmi_test,RSF Score
5,1,0,1,0,0,1,1,0,0,1,85,1,20,0
7,0,1,0,0,0,1,1,0,0,1,156,1,38,0
8,0,1,0,0,1,1,1,0,0,1,275,1,8,0
17,1,0,1,0,0,0,1,0,1,0,47,1,25,0
19,1,0,1,0,1,0,0,0,1,0,32,1,22,1


In [11]:
# Evaluation on test data

df = data_fit_test.loc[data_fit_test['RSF Score']==1, cols_basic+confounders+[treatment]].copy()
HR = Cox_IPTW(df, treatment)
print('On test data, treatment HR for patients with RSF Score High = %s' % HR)

df = data_fit_test.loc[data_fit_test['RSF Score']==0, cols_basic+confounders+[treatment]].copy()
HR = Cox_IPTW(df, treatment)
print('On test data, treatment HR for patients with RSF Score High = %s' % HR)

On test data, treatment HR for patients with RSF Score High = 0.75 (0.60, 0.94)
On test data, treatment HR for patients with RSF Score High = 1.15 (1.00, 1.33)
